## Open Government Data, provided by **OpenDataZurich**
*Autogenerated Python starter code for data set with identifier* **geo__gdp__inventar_der_schuetzenswerten_gaerten_und_anlagen_von_kommunaler_bedeutung_der_stadt_**

## Dataset
# **(GDP) Inventar der schützenswerten Gärten und Anlagen von kommunaler Bedeutung der Stadt Zürich**

## Description

Im GDP-Inventar werden die schützenswerten bzw. potentiell schutzwürdigen Gärten und Anlagen von kommunaler Bedeutung der Stadt Zürich dargestellt.

**Zweck**: Der Fachbereich Gartendenkmalpflege bei Grün Stadt Zürich führt basierend auf dem Gesetz über die Raumplanung und das öffentliche Baurecht (Planungs- und Baugesetz, PBG) sowie aufgrund des Stadtratsbeschlusses 873 vom 26. März 1986 das Inventar der schützenswerten Gärten und Anlagen von kommunaler Bedeutung. Dieses wurde mit dem Stadtratsbeschluss 2321 vom 19. Juli 1989 festgesetzt.

**Genereller Hinweis zum Geodatensatz:**


Weitere Informationen zu diesem Datensatz finden sie unter [«**(GDP) Inventar der schützenswerten Gärten und Anlagen von kommunaler Bedeutung der Stadt Zürich»** auf Geocat.ch](https://www.geocat.ch/geonetwork/srv/ger/catalog.search#/metadata/efbeacb0-f492-3fe6-6cd3-63127a09bf0b).				
			   

**Informationen zum Datensatz:**

Zusammenfassung:
Dargestellt werden die schützenswerten bzw. potentiell schutzwürdigen Gärten und Anlagen von kommunaler Bedeutung der Stadt Zürich. Das Inventar wurde durch Beschluss des Stadtrates 1989 in Kraft gesetzt. Neuaufnahmen, Unterschutzstellungen und Entlassungen werden nachgeführt. Das Inventar ist ebenfalls ab Mitte August 2010 über die Katasterauskunft  der Stadt Zürich im Internet öffentlich zugänglich.  

Hinweis:
Vor einer Sanierung, Renovation, Umgestaltung oder Abbruch müssen das Amt für Baube-willigungen oder die Gartendenkmalpflege angefragt werden. Wurde ein Garten oder eine Anlage formell vom Stadtrat unter Schutz gestellt ist vor einer angestrebten Veränderung des Objekts rechtzeitig  mit der Gartendenkmalpflege Kontakt aufzunehmen.

**Datenerfassung:**

Das Inventar wurde ursprünglich nur in einer Filemaker-Datenbank nachgeführt, seit 2001 jedoch auch als Geodatenbestand. Die Verbindung von Filemaker und GIS erfolgt über die Adresse und/oder Parzellen-Nummer. Da   der grösste Teil der Objekte auf den Liegenschaften der Amtlichen Vermessung basiert, ist die theoretische Standortgenauigkeit sehr hoch. In der Regel ist jedoch nur ein Teil einer Liegenschaft inventarisiert und/oder geschützt, so dass die relevante Standortsgenauigkeit gering ist.

**Datengrundlage:**

Orthophotos der Stadt Zürich (Digitalisier-Grundlage)
Übersichtsplan der Stadt Zürich (Digitalisier-Grundlage)
Daten der amtlichen Vermesssung (Digitalisier-Grundlage)


**Weitere Informationen zu diesem Thema:**

Finden Sie unter diesem [**Link**](http://www.stadt-zuerich.ch/ted/de/index/gsz/planung_u_bau/inventare_und_grundlagen/gartendenkmalpflege-inventar.html)

**Statisches Vorschaubild:**

![BildText](https://www.gis.stadt-zuerich.ch/zueriplan_docs/geocat/efbeacb0-f492-3fe6-6cd3-63127a09bf0b.png)



## Data set links

[Direct link by OpenDataZurich for dataset](https://data.stadt-zuerich.ch/dataset/geo__gdp__inventar_der_schuetzenswerten_gaerten_und_anlagen_von_kommunaler_bedeutung_der_stadt_)

https://www.stadt-zuerich.ch/geodaten/download/_GDP__Inventar_der_schuetzenswerten_Gaerten_und_Anlagen_von_kommunaler_Bedeutung_der_Stadt_Zuerich?format=geojson_link<br>
{{ DATASHOP_LINK_ORGANIZATION }}

## Metadata
- **Publisher** `Gartendenkmalpflege, Grün Stadt Zürich, Tiefbau- und Entsorgungsdepartement`
- **Maintainer** `Open Data Zürich`
- **Maintainer_email** `opendata@zuerich.ch`
- **Keywords** `Umwelt`
- **Tags** `['baum', 'bestattung', 'freiraum', 'garten', 'gdp', 'geodaten', 'geoportal', 'grunflache', 'polygondaten', 'schutz', 'stzh', 'vektordaten']`
- **Metadata_created** `2023-11-06T03:21:29.257790`
- **Metadata_modified** `2025-02-18T09:26:39.862253`


## Imports and helper functions

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

params = {
    'text.color': (0.25, 0.25, 0.25),
    'figure.figsize': [18, 6],
   }

plt.rcParams.update(params)

import pandas as pd 

import json
import xml.etree.ElementTree as ET
import requests
import geopandas as gpd
import re

In [ ]:
# helper function for reading datasets with proper separator
def get_dataset(url):
    # get URL ofs WFS service
    dataset_identifier = re.search(r'\/([^\/\?]+)\?', url).group(1)
    url_geoportal = f"https://www.ogd.stadt-zuerich.ch/wfs/geoportal/{dataset_identifier}"
    print("Getting available layers from:", url_geoportal)
    
    # Parameter for GetCapabilities
    params = {
        "service": "WFS",
        "version": "1.1.0",
        "request": "GetCapabilities"
    }
    
    # send GetCapabilities
    response = requests.get(url_geoportal, params=params)

    # parse XML answer
    root = ET.fromstring(response.content)

    # define Namespace 
    namespace = {'wfs': 'http://www.opengis.net/wfs'}
    
    # Exctract available layers
    layers = [feature_type.find('wfs:Name', namespace).text for feature_type in root.findall('.//wfs:FeatureType', namespace)]
    
    print("Available layers:", layers)
    print("First layer is set as default. To chose another layer set it as typename in the get_dataset() function.")

    # set first layer als typename
    typename = layers[0]
    print("Chosen typename:", typename)

    # Parameter GetFeature request
    params = {
        "service": "WFS",
        "version": "1.1.0",
        "request": "GetFeature",
        "typename": typename,
        "outputFormat": "application/json"
    }
    
    # GetFeature request
    response = requests.get(url_geoportal, params=params)

    # Load GeoJSON in GeoDataFrame
    gdf = gpd.read_file(json.dumps(response.json()))
    return gdf

## Load data

- The dataset has **`{{ DISTRIBUTION_COUNT }}` distribution(s)** in CSV format.
- All available CSV distributions are listed below and can be read into a pandas dataframe.

In [ ]:
# name: _GDP__Inventar_der_schuetzenswerten_Gaerten_und_Anlagen_von_kommunaler_Bedeutung_der_Stadt_Zuerich.json (GeoJSON-Services)
# filename: nan
# format: JSON
# url: https://www.stadt-zuerich.ch/geodaten/download/_GDP__Inventar_der_schuetzenswerten_Gaerten_und_Anlagen_von_kommunaler_Bedeutung_der_Stadt_Zuerich?format=geojson_link
# id: 61681bfd-373e-49bd-b6b0-7c449e512cb4
# resource_type: api
# package_id: 336506c8-4f50-4d0a-b5e4-ea3f6aa3251c

gdf = get_dataset('https://www.stadt-zuerich.ch/geodaten/download/_GDP__Inventar_der_schuetzenswerten_Gaerten_und_Anlagen_von_kommunaler_Bedeutung_der_Stadt_Zuerich?format=geojson_link')


## Analyze data

In [ ]:
gdf.plot()

In [ ]:
# drop columns that have no values
gdf.dropna(how='all', axis=1, inplace=True)

In [ ]:
print(f'The dataset has {gdf.shape[0]:,.0f} rows (observations) and {gdf.shape[1]:,.0f} columns (variables).')
print(f'There seem to be {gdf.duplicated().sum()} exact duplicates in the data.')

In [ ]:
gdf.info(memory_usage='deep', verbose=True)

In [ ]:
gdf.head()

In [ ]:
# display a small random sample transposed in order to see all variables
gdf.sample(3).T

In [ ]:
# describe non-numerical features
try:
    with pd.option_context('display.float_format', '{:,.2f}'.format):
        display(gdf.describe(exclude='number'))
except:
    print("No categorical data in dataset.")

In [ ]:
# describe numerical features
try:
    with pd.option_context('display.float_format', '{:,.2f}'.format):
        display(gdf.describe(include='number'))
except:
    print("No numercial data in dataset.")

In [ ]:
# plot a histogram for each numerical feature
try:
    gdf.hist(bins=25, rwidth=.9)
    plt.tight_layout()
    plt.show()
except:
    print("No numercial data to plot.") 

In [ ]:
# continue your code here...

**Contact**: opendata@zuerich.ch